In [3]:
!pip install transformers datasets --quiet
from transformers import pipeline
from datasets import load_dataset
from sklearn.metrics import accuracy_score, f1_score
import pandas as pd

print("✅ Libraries loaded successfully.")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 18.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-nvrtc-cu12

In [4]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_name = "google/flan-t5-base"

print(f"⏳ Loading FLAN-T5 model: {model_name}")
generator = pipeline("text2text-generation", model=model_name, tokenizer=model_name)
print("✅ FLAN-T5 loaded and ready.")

⏳ Loading FLAN-T5 model: google/flan-t5-base


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

...

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

Device set to use cuda:0


✅ FLAN-T5 loaded and ready.


In [5]:
print("⏳ Loading MRPC and QQP datasets...")

mrpc = load_dataset("glue", "mrpc")
qqp = load_dataset("glue", "qqp")

# Use small samples for quick CPU evaluation
mrpc_sample = mrpc["validation"].select(range(100))
qqp_sample = qqp["validation"].select(range(100))

print(f"✅ MRPC samples: {len(mrpc_sample)}")
print(f"✅ QQP samples: {len(qqp_sample)}")

⏳ Loading MRPC and QQP datasets...


README.md:   0%|          | 0.00/35.3k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/649k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/75.7k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/308k [00:00<?, ?B/s]

...

test-00000-of-00001.parquet:   0%|          | 0.00/36.7M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/363846 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/40430 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/390965 [00:00<?, ? examples/s]

✅ MRPC samples: 100
✅ QQP samples: 100


In [6]:
def format_prompt(q1, q2):
    return f"""Determine if the two sentences express the same meaning.

Sentence 1: {q1}
Sentence 2: {q2}

Answer:"""

print("✅ Prompt formatting function ready.")

✅ Prompt formatting function ready.


In [8]:
def get_prediction(sentence1, sentence2):
    prompt = format_prompt(sentence1, sentence2)
    output = generator(prompt, max_new_tokens=5)[0]['generated_text'].strip().lower()
    return 1 if "yes" in output else 0

def evaluate_on_dataset(dataset, name=""):
    print(f"\n🚀 Evaluating on {name.upper()}...")
    preds, labels = [], []

    for row in dataset:
        if name.lower() == "qqp":
            s1, s2 = row['question1'], row['question2']
        else:
            s1, s2 = row['sentence1'], row['sentence2']
        label = row['label']

        pred = get_prediction(s1, s2)
        preds.append(pred)
        labels.append(label)

    acc = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds, average='macro')

    print(f"📊 {name.upper()} Results — Accuracy: {acc:.4f}, Macro F1: {f1:.4f}")
    return acc, f1


# Run evaluation
acc_mrpc, f1_mrpc = evaluate_on_dataset(mrpc_sample, "MRPC")
acc_qqp, f1_qqp = evaluate_on_dataset(qqp_sample, "QQP")



🚀 Evaluating on MRPC...
📊 MRPC Results — Accuracy: 0.7500, Macro F1: 0.7270

🚀 Evaluating on QQP...
📊 QQP Results — Accuracy: 0.8200, Macro F1: 0.7995


In [9]:
print("\n✅ FINAL SUMMARY")
print(f"MRPC — Accuracy: {acc_mrpc:.4f}, Macro F1: {f1_mrpc:.4f}")
print(f"QQP  — Accuracy: {acc_qqp:.4f}, Macro F1: {f1_qqp:.4f}")


✅ FINAL SUMMARY
MRPC — Accuracy: 0.7500, Macro F1: 0.7270
QQP  — Accuracy: 0.8200, Macro F1: 0.7995
